In [1]:
import os
import json 
import numpy as np
import pandas as pd
import tensorflow as tf

from nltk_utils import tokenize, stem, bag_of_words
from speech import get_transcript, speak

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vrush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Preprocessing

In [2]:
ignore_words = ["?", "i", "it", "a", "!", "me", "you", "'s", "does", "of", "do", "my", "are", "you"]

In [3]:
# get tags and set of all words from training data
# xy contains sample prompt and its expected output category


with open("intents.json", "r") as f:
    intents = json.load(f)

tags = []
all_words = []
xy = []

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    
    for pattern in intent["patterns"]:
        
        words = tokenize(pattern)
        l1 = [stem(w) for w in words if w.lower() not in ignore_words]
        
        all_words.extend(l1)
        xy.append((l1, tag))

all_words = sorted(set(all_words))
np.random.shuffle(xy)

In [4]:
print(tags)
print(all_words)
print(xy)

['unknown', 'greeting', 'goodbye', 'thanks', 'items', 'payments', 'delivery', 'funny']
['appreci', 'avail', 'buy', 'bye', 'can', 'card', 'cash', 'credit', 'deliv', 'deliveri', 'expect', 'funni', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'is', 'item', 'joke', 'know', 'later', 'laugh', 'lot', 'make', 'okay', 'on', 'packag', 'pay', 'payment', 'paypal', 'product', 'realli', 'reciev', 'sell', 'ship', 'should', 'someth', 'thank', 'that', 'the', 'time', 'up', 'use', 'what', 'when', 'will']
[(['hi'], 'greeting'), (['what', 'up'], 'greeting'), (['ship'], 'delivery'), (['okay'], 'thanks'), (['funni', 'joke'], 'funny'), (['deliv'], 'delivery'), (['thank'], 'thanks'), (['hello'], 'greeting'), (['thank'], 'thanks'), (['someth', 'funni'], 'funny'), (['later'], 'goodbye'), (['that', 'help'], 'thanks'), (['bye'], 'goodbye'), (['what', 'is', 'the', 'expect', 'deliveri', 'time'], 'delivery'), (['when', 'will', 'reciev', 'packag'], 'delivery'), (['sell'], 'items'), (['product'], 'items'), ([

In [5]:
df = pd.DataFrame(xy)
df.head()

,0,1
0,[hi],greeting
1,"[what, up]",greeting
2,[ship],delivery
3,[okay],thanks
4,"[funni, joke]",funny


In [6]:
df[df[1] == "delivery"]

,0,1
2,[ship],delivery
5,[deliv],delivery
13,"[what, is, the, expect, deliveri, time]",delivery
14,"[when, will, reciev, packag]",delivery


In [7]:
# Prepare training set

X_train = []
y_train = []

for sentence, tag in xy:
    bag = bag_of_words(sentence, all_words)
    X_train.append(bag)

    label = bag_of_words(tag, tags)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [8]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
y_train

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 

# Model Training

In [10]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=16, input_shape = np.shape(all_words), activation='relu'))
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

ann.add(tf.keras.layers.Dense(units=len(tags) , activation='softmax'))

ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy']) 

d:\AI\Projects\Realtime Voice-enabled AI Chatbot\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
ann.fit(X_train, y_train, batch_size=6, epochs = 200)

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0415 - loss: 2.0963      
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - accuracy: 0.0886 - loss: 2.0842   
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2289 - loss: 2.0669 
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2216 - loss: 2.0525 
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1879 - loss: 2.0405     
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2324 - loss: 2.0504 
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2865 - loss: 2.0088 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3768 - loss: 2.0169 
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5331 - loss: 1.9779 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4576 - loss: 1.9777 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4703 - loss: 1.9677 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy:

In [12]:
# Tag index of input sentence

# input needs to be tokenized, stemmed and vectorized
input_1 = "hello"

# ann.predict(bag_of_words(input_1, all_words).reshape(1, -1))
np.argmax(ann.predict(bag_of_words(input_1, all_words).reshape(1, -1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


1

# Response

In [13]:
# Respond based on given set of training responses

def respond(sentence):

    '''
    input : 
        sentence : input prompt from user

    output : 
        response : text from given set of responses in training data
        tag :      tag of input prompt
    '''

    words = tokenize(sentence)
    sentence = [stem(w) for w in words if w not in ignore_words]
    bag = bag_of_words(sentence, all_words).reshape(1, -1)
    
    probs = ann.predict(bag)

    if max(probs[0]) < 0.3 :
        max_idx = 0
    else :
        max_idx = np.argmax(probs)
    response = np.random.choice(intents['intents'][max_idx]['responses'])
    tag = intents['intents'][max_idx]['tag']

    return response, tag


In [14]:
input_2 = "how can i pay ?"

respond(input_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


('We accept VISA, Mastercard and Paypal', 'payments')

# Main

In [15]:
lang = 'english'

while True :

    text, correct = get_transcript(lang)

    if correct :

        print("You : ", text)
        response, tag = respond(text)

        print("Chatbot : ", response)
        speak(response)

        if tag == 'goodbye' :
            break
    else :

        speak(text)


Listening ......

Could not understand audio

Listening ......

You :  bye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Chatbot :  Bye! Come back again soon.
